**Libraries**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

**Data Extraction**

In [ ]:
base = "https://ackodrive.com"
# Replace the car name to derieve data from cars.
car = "renault+cars"
city = "Mumbai"
url = f"https://ackodrive.com/collection/{car}/?city={city}"

headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

car_cards = soup.find_all("div", class_="BuyCarCard_card__AsGXF")
print("Cars Found:", len(car_cards))

data = []

for car in car_cards:

    # Title
    title_tag = car.find("a", class_="BuyCarCard_carName__SAJVh")
    title = title_tag.text.strip() if title_tag else None

    # Car page link
    link = base + title_tag.get("href") if title_tag else None

    # Price
    price_tag = car.find("p", class_="BuyCarCard_priceRange__2q8tm")
    price = price_tag.text.strip() if price_tag else None


    fuel = None
    transmission = None

    if link:
        detail_res = requests.get(link, headers=headers)
        detail_soup = BeautifulSoup(detail_res.text, "html.parser")

        # Fuel and Transmission
        spec = detail_soup.find("p", class_="SpecsAndFeatureCard_card__text__wxvgC")

        if spec:
            parts = [p.strip() for p in spec.text.split("•")]
            fuel = parts[0] if len(parts) > 0 else None
            transmission = parts[1] if len(parts) > 1 else None

        time.sleep(1)


    data.append({
        "Title": title,
        "Price": price,
        "Fuel Type": fuel,
        "Transmission": transmission,
        "Location": city,
    })

Cars Found: 5


**Data Cleaning**

In [ ]:
df = pd.DataFrame(data)

df["Title"] = df["Title"].str.strip()
df.replace({"": None}, inplace=True)

df.index = df.index + 1

print(df)



                        Title                   Price Fuel Type Transmission  \
1   Renault Kiger (2021-2025)  ₹7.2 lakh – ₹13.3 lakh    Petrol          CNG   
2  Renault Triber (2019-2025)  ₹7.2 lakh – ₹10.3 lakh    Petrol          CNG   
3                Renault Kwid   ₹4.9 lakh – ₹7.4 lakh    Petrol          CNG   
4               Renault Kiger  ₹6.5 lakh – ₹12.1 lakh    Petrol          CNG   
5              Renault Triber   ₹6.5 lakh – ₹9.8 lakh    Petrol          CNG   

  Location  
1   Mumbai  
2   Mumbai  
3   Mumbai  
4   Mumbai  
5   Mumbai  


**Export to CSV**

In [ ]:
df.to_csv("Mini_Project-Group_F.csv", index=False)
print("CSV file saved as Mini_Project-Group_F.csv")

CSV file saved as Mini_Project-Group_F.csv
